In [55]:
from nvi_etl.utilities import fix_parcel_id
import pandas as pd
import geopandas as gpd
from shapely import wkb

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\mike\\Desktop\\2_responsibilities\\nvi_etl\\secondary_ipds\\config.toml'

In [ ]:
tax = pd.read_csv("V:/IPDS/Wayne County Tax Foreclosures/Data/2024/Prepared/wcto_foreclosed_05092024.csv")
detodp = pd.read_csv("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/detodp_assessor_20240205.csv")


tax = tax[tax["CITY"] == 'DETROIT']

# Convert WKB to Geometry
detodp["geometry"] = detodp["geom"].apply(wkb.loads)

# Convert to a GeoDataFrame
detodp_gdf = gpd.GeoDataFrame(detodp, geometry="geometry", crs="EPSG:4326")

# Drop the old WKB column if not needed
detodp_gdf.drop(columns=["geom"], inplace=True)


# Apply the function to the PARCEL_ID column
tax = tax.dropna(subset=["PARCEL_ID"])
tax["PARCEL_ID"] = tax["PARCEL_ID"].astype(str).apply(fix_parcel_id)

citywide = (
    tax.aggregate(Detroit=("PARCEL_ID", "count"))
    .assign(geo_type="citywide", geography="Detroit", total_properties=len(detodp))
    .rename(columns={"PARCEL_ID": "num_foreclosures"})
    .assign(foreclosure_rate=lambda df: 100 * df["num_foreclosures"] / df["total_properties"])
    .reset_index().drop("index", axis=1)
)


nvi_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/NVI Zones/nvi_neighborhood_zones_temp_2025.shp")
detroit_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp")
cd_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Detroit_City_Council_Districts_2026/Detroit_City_Council_Districts_2026.shp")

merged = pd.merge(tax, detodp, left_on="PARCEL_ID", right_on="parcel_num", how="inner")


# Convert WKB to Geometry
merged["geometry"] = merged["geom"].apply(wkb.loads)

# Convert to a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged, geometry="geometry", crs="EPSG:4326")

# Drop the old WKB column if not needed
merged_gdf.drop(columns=["geom"], inplace=True)

# NVI ZONES

merged_nvi = merged_gdf.to_crs(nvi_shp.crs)
merged_with_nvi = gpd.sjoin(merged_nvi, nvi_shp, how="left", predicate="within")
detodp_nvi= detodp_gdf.to_crs(nvi_shp.crs)
detodp_with_nvi = gpd.sjoin(detodp_nvi, nvi_shp, how="left", predicate="within")

# Total number of properties in foreclosure, grouped by district_n and zone_id
num_foreclosures_nvi = merged_with_nvi.groupby(["zone_id"]).size().reset_index(name="num_foreclosures")

# Total number of properties (from detodp with geography), grouped by district_n and zone_id
total_properties_nvi = detodp_with_nvi.groupby(["zone_id"]).size().reset_index(name="total_properties")

foreclosure_rate_nvi = num_foreclosures_nvi.merge(total_properties_nvi, on="zone_id", how="left")
foreclosure_rate_nvi["foreclosure_rate"] = (foreclosure_rate_nvi["num_foreclosures"] / foreclosure_rate_nvi["total_properties"]) * 100

nvi_zones = foreclosure_rate_nvi.assign(geo_type="zone").rename(columns={"zone_id": "geography"})

## COUNCIL DISTRICTS
merged_cd = merged_gdf.to_crs(cd_shp.crs)
merged_with_cd = gpd.sjoin(merged_cd, cd_shp, how="left", predicate="within")
detodp_cd= detodp_gdf.to_crs(cd_shp.crs)
detodp_with_cd = gpd.sjoin(detodp_cd, cd_shp, how="left", predicate="within")

# Total number of properties in foreclosure, grouped by district_n and zone_id
num_foreclosures_cd = merged_with_cd.groupby("district_n").size().reset_index(name="num_foreclosures")

# Total number of properties (from detodp with geography), grouped by district_n and zone_id
total_properties_cd = detodp_with_cd.groupby("district_n").size().reset_index(name="total_properties")

foreclosure_rate_cd = num_foreclosures_cd.merge(total_properties_cd, on="district_n", how="left")
foreclosure_rate_cd["foreclosure_rate"] = (foreclosure_rate_cd["num_foreclosures"] / foreclosure_rate_cd["total_properties"]) * 100

council_districts = (
    foreclosure_rate_cd
    .assign(geo_type="district")
    .rename(columns={"district_n": "geography"})
    .astype({"geography": "int"})
    .astype({"geography": "str"})
)

pd.concat([
    citywide,
    nvi_zones,
    council_districts,
])

C:\Users\mike\AppData\Local\Temp\ipykernel_21736\3095674299.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  detodp = pd.read_csv("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/detodp_assessor_20240205.csv")


,num_foreclosures,geo_type,geography,total_properties,foreclosure_rate
0,2187,citywide,Detroit,381265,0.573617
0,80,zone,1a,16870,0.474215
1,68,zone,1b,15165,0.448401
2,97,zone,1c,16990,0.570924
3,89,zone,2a,18422,0.483118
4,34,zone,2b,10570,0.321665
5,137,zone,2c,15526,0.882391
6,197,zone,3a,28045,0.702443
7,148,zone,3b,16550,0.894260
8,95,zone,3c,22114,0.429592
